In [3]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [4]:
client : OpenAI = OpenAI()

In [5]:


def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [6]:


def show_dict(obj):
    display(obj.model_dump())

### Step 1: Create an Assistant

In [7]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
    name="Travel Assitant",
    instructions="You are a personal Travel Assistant. You should reply to user query about the asked location with few bullet points.",
    #tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

show_json(assistant)
show_dict(assistant)

{'id': 'asst_cXFUQA0NxyXcNvxFhqWMSmBf',
 'created_at': 1704797864,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal Travel Assistant. You should reply to user query about the asked location with few bullet points.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Travel Assitant',
 'object': 'assistant',
 'tools': []}

{'id': 'asst_cXFUQA0NxyXcNvxFhqWMSmBf',
 'created_at': 1704797864,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal Travel Assistant. You should reply to user query about the asked location with few bullet points.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Travel Assitant',
 'object': 'assistant',
 'tools': []}

### Step 2: Create a Thread

In [9]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()


### Step 3: Add a Message to a Thread

In [10]:
from openai.types.beta.threads.thread_message import ThreadMessage

message : ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to travel to Paris,France. Can you give me five best places to visit in Paris?"
)

show_json(message)

{'id': 'msg_Wtb61A3kzt2tZdcXuxANckE8',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to travel to Paris,France. Can you give me five best places to visit in Paris?'},
   'type': 'text'}],
 'created_at': 1704797921,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_Kp9Fivej1MGWBqgD09HSsffT'}

### Step 4: Run the Assistant

In [11]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="Please address the user as Jane Doe. The user has a premium account."
)

show_json(run)


{'id': 'run_Hv2Rf8Mv6LHDCKfjQRFsrwO6',
 'assistant_id': 'asst_cXFUQA0NxyXcNvxFhqWMSmBf',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704797932,
 'expires_at': 1704798532,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal Travel Assistant. You should reply to user query about the asked location with few bullet points.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_Kp9Fivej1MGWBqgD09HSsffT',
 'tools': []}

### Step 5: Check the Run status

In [12]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)


#show_dict(run)
run.model_dump()

{'id': 'run_Hv2Rf8Mv6LHDCKfjQRFsrwO6',
 'assistant_id': 'asst_cXFUQA0NxyXcNvxFhqWMSmBf',
 'cancelled_at': None,
 'completed_at': 1704797937,
 'created_at': 1704797932,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal Travel Assistant. You should reply to user query about the asked location with few bullet points.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1704797932,
 'status': 'completed',
 'thread_id': 'thread_Kp9Fivej1MGWBqgD09HSsffT',
 'tools': []}

### Step 6: Display the Assistant's Response

In [13]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  #order="asc"
)

msg = client.beta.threads.messages.retrieve(
  thread_id= thread.id,
  message_id= message.id,
)

show_json(messages)
show_json(msg)

for m in reversed(messages.data):
   print(m.role + ": " + m.content[0].text.value)

# for m in messages.data:
#    print(m.role + ": " + m.content[0].text.value)


{'data': [{'id': 'msg_ZA0vATbPcL7eF79wIg2DCMtc',
   'assistant_id': 'asst_cXFUQA0NxyXcNvxFhqWMSmBf',
   'content': [{'text': {'annotations': [],
      'value': '- Eiffel Tower: Iconic landmark with breathtaking views of the city\n- Louvre Museum: Home to famous works of art including the Mona Lisa\n- Notre-Dame Cathedral: Magnificent gothic architecture and history\n- Montmartre: Charming district known for its artsy vibe and Sacré-Cœur Basilica\n- Seine River: Enjoy a relaxing boat cruise and take in the city from a different perspective'},
     'type': 'text'}],
   'created_at': 1704797933,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_Hv2Rf8Mv6LHDCKfjQRFsrwO6',
   'thread_id': 'thread_Kp9Fivej1MGWBqgD09HSsffT'},
  {'id': 'msg_Wtb61A3kzt2tZdcXuxANckE8',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to travel to Paris,France. Can you give me five best places to visit in Paris

{'id': 'msg_Wtb61A3kzt2tZdcXuxANckE8',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to travel to Paris,France. Can you give me five best places to visit in Paris?'},
   'type': 'text'}],
 'created_at': 1704797921,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_Kp9Fivej1MGWBqgD09HSsffT'}

user: I need to travel to Paris,France. Can you give me five best places to visit in Paris?
assistant: - Eiffel Tower: Iconic landmark with breathtaking views of the city
- Louvre Museum: Home to famous works of art including the Mona Lisa
- Notre-Dame Cathedral: Magnificent gothic architecture and history
- Montmartre: Charming district known for its artsy vibe and Sacré-Cœur Basilica
- Seine River: Enjoy a relaxing boat cruise and take in the city from a different perspective
